In [5]:
# RESULT WITH A SINGLE MODEL


# get one trajectory from target experiment
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

directory = '/scratch/a.cossu/results_single/rho_10.0_nhid_2_timesteps_3000'
seq_len = 2000  # timesteps - washout
n_init_states = 1000
n_inp = 1
n_hid = 2
pca = np.load(os.path.join(directory, 'pca_result.npy'))  # (n_init_states*timesteps, pca_dim)
trajectories = []
for i in range(0, pca.shape[0], seq_len):
    trajectories.append(pca[i:i+seq_len])
assert(len(trajectories) == n_init_states) 

filename = "lyapunov_rnn.dat"

input_signals = np.load(os.path.join(directory, 'u_timeseries.npy'))

# move files from the saved directory to the current directory
shutil.move(os.path.join(directory, "W.csv"), "W.csv")
shutil.move(os.path.join(directory, "V.csv"), "V.csv")
shutil.move(os.path.join(directory, "b.csv"), "b.csv")

kds = []
for i, traj in enumerate(tqdm(trajectories)):
    input_signal = np.expand_dims(input_signals[i], axis=-1) # (seq_len, 1)
    np.savetxt("u_timeseries.csv", input_signal, delimiter=",", fmt="%.6f")
    np.savetxt("h_traj.csv", traj, delimiter=',', fmt="%.6f")

    # call rnn_lyap with parameters n_hid, seq_len, n_inp
    os.system(f"./rnn_lyap {n_hid} {seq_len} {n_inp}")

    data = np.loadtxt(filename)
    # Extract time (first column) and exponents (remaining columns)
    time = data[:, 0]
    lyapunov_exponents = data[:, 1:]


    # Plot each Lyapunov exponent as a function of time
    # plt.figure(figsize=(10, 6))
    # # plot the zero line for reference in black dashed line
    # plt.plot(time, np.zeros_like(time), color='black', linestyle='--', linewidth=1)
    # for i in range(lyapunov_exponents.shape[1]):
    #     plt.plot(time, lyapunov_exponents[:, i], label=f"LE {i+1}")

    # plt.xlabel("Time")
    # plt.ylabel("Lyapunov Exponents")
    # plt.title("Convergence of Lyapunov Exponents in RNN")
    # plt.legend()
    # plt.grid(True, linestyle='--', alpha=0.6)
    # plt.tight_layout()
    # plt.show()

    best_approx_lyaps = lyapunov_exponents[-1, :]
    # print(best_approx_lyaps)

    # Example: lyapunov_exponents.shape is (N,)
    # Make sure the spectrum is sorted in descending order
    best_approx_lyaps = np.sort(best_approx_lyaps)[::-1]

    if np.any(best_approx_lyaps > 0):
        # Cumulative sum
        cumsum = np.cumsum(best_approx_lyaps)

        # Find j: largest index where cumulative sum is still >= 0
        j = np.where(cumsum >= 0)[0][-1]  # last index satisfying sum >= 0

        # print("j (number of non-negative LEs):", j + 1)

        # Compute Kaplan-Yorke dimension
        if j + 1 < len(best_approx_lyaps):
            D_KY = j + 1 + cumsum[j] / abs(best_approx_lyaps[j + 1])
        else:
            # All exponents non-negative 
            D_KY = len(best_approx_lyaps)
    else:
        # All exponents are non-positive
        D_KY = 0
    # print("******************")
    #print("Kaplan-Yorke dimension:", D_KY)
    # print("******************")
    kds.append(D_KY)

print(f"Kaplan-Yorke dimension over trajectories: {np.mean(kds)} ± {np.std(kds)}")

# move files back to the saved directory
shutil.move("W.csv", os.path.join(directory, "W.csv"))
shutil.move("V.csv", os.path.join(directory, "V.csv"))
shutil.move("b.csv", os.path.join(directory, "b.csv"))

100%|██████████| 1000/1000 [00:11<00:00, 89.40it/s]

Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0


'/scratch/a.cossu/results_single/rho_10.0_nhid_2_timesteps_3000/b.csv'

In [8]:
# RESULT WITH A COLLECTIVE OF MODELS


# get one trajectory from target experiment
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import shutil

directory = '/scratch/a.cossu/results_collective/mod2_rho_10.0_nhid_2_timesteps_3000_inpscaling_1.0'
seq_len = 2000  # timesteps - washout
n_init_states = 1000
n_hid = 2  # TODO: the hidden size must be equal to the dimension of the trajectories, so not in the PCA dim?
n_inp = n_hid
n_modules = 2

input_signal = torch.load(os.path.join(directory, f"input_signals.pt"))

for i in range(n_modules):
    print(f"Processing module {i}")
    pca = np.load(os.path.join(directory, f'pca_result_{i}.npy'))  # (n_init_states*timesteps, pca_dim)
    trajectories = []

    for tt in range(0, pca.shape[0], seq_len):
        trajectories.append(pca[tt:tt+seq_len])
    assert(len(trajectories) == n_init_states) 

    filename = "lyapunov_rnn.dat"

    # move files from the saved directory to the current directory
    shutil.move(os.path.join(directory, f"W_{i}.csv"), f"W.csv")
    shutil.move(os.path.join(directory, f"V_{i}.csv"), f"V.csv")
    shutil.move(os.path.join(directory, f"b_{i}.csv"), f"b.csv")

    kds = []
    for t, traj in enumerate(tqdm(trajectories)):
        np.savetxt(os.path.join("u_timeseries.csv"), input_signal[i][t].cpu().numpy(), delimiter=",", fmt="%.6f")
        np.savetxt("h_traj.csv", traj, delimiter=',', fmt="%.6f")

        # call rnn_lyap with parameters n_hid, seq_len, n_inp
        os.system(f"./rnn_lyap {n_hid} {seq_len} {n_inp}")

        data = np.loadtxt(filename)
        # Extract time (first column) and exponents (remaining columns)
        time = data[:, 0]
        lyapunov_exponents = data[:, 1:]


        # Plot each Lyapunov exponent as a function of time
        # plt.figure(figsize=(10, 6))
        # # plot the zero line for reference in black dashed line
        # plt.plot(time, np.zeros_like(time), color='black', linestyle='--', linewidth=1)
        # for i in range(lyapunov_exponents.shape[1]):
        #     plt.plot(time, lyapunov_exponents[:, i], label=f"LE {i+1}")

        # plt.xlabel("Time")
        # plt.ylabel("Lyapunov Exponents")
        # plt.title("Convergence of Lyapunov Exponents in RNN")
        # plt.legend()
        # plt.grid(True, linestyle='--', alpha=0.6)
        # plt.tight_layout()
        # plt.show()

        best_approx_lyaps = lyapunov_exponents[-1, :]
        # print(best_approx_lyaps)

        # Example: lyapunov_exponents.shape is (N,)
        # Make sure the spectrum is sorted in descending order
        best_approx_lyaps = np.sort(best_approx_lyaps)[::-1]

        if np.any(best_approx_lyaps > 0):
            # Cumulative sum
            cumsum = np.cumsum(best_approx_lyaps)

            # Find j: largest index where cumulative sum is still >= 0
            j = np.where(cumsum >= 0)[0][-1]  # last index satisfying sum >= 0

            # print("j (number of non-negative LEs):", j + 1)

            # Compute Kaplan-Yorke dimension
            if j + 1 < len(best_approx_lyaps):
                D_KY = j + 1 + cumsum[j] / abs(best_approx_lyaps[j + 1])
            else:
                # All exponents non-negative 
                D_KY = len(best_approx_lyaps)
        else:
            # All exponents are non-positive
            D_KY = 0
        # print("******************")
        #print("Kaplan-Yorke dimension:", D_KY)
        # print("******************")
        kds.append(D_KY)

    print(f"Kaplan-Yorke dimension over trajectories: {np.mean(kds)} ± {np.std(kds)}")

    # move files back to the saved directory
    shutil.move("W.csv", os.path.join(directory, f"W_{i}.csv"))
    shutil.move("V.csv", os.path.join(directory, f"V_{i}.csv"))
    shutil.move("b.csv", os.path.join(directory, f"b_{i}.csv"))
    print("=========================================")

Processing module 0


100%|██████████| 1000/1000 [00:08<00:00, 118.63it/s]


Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
Processing module 1


100%|██████████| 1000/1000 [00:12<00:00, 82.46it/s]

Kaplan-Yorke dimension over trajectories: 0.0 ± 0.0
